In [1]:
import Pkg
Pkg.activate("../lecture-julia.notebooks")
using Expectations, ForwardDiff, QuadGK, LinearAlgebra, FastGaussQuadrature

  Activating project at `~/ECON622/lecture-julia.notebooks`


## Adaptive Quadrature
Gauss-Kronod is very precise, changes step sizes

In [2]:
f(x) = x^2
x_min = 0.0
x_max = 1.0
x = x_min:0.01:x_max
x_array = Array(x)  # turns into an array.  Doesn't need to be uniform anymore
@show quadgk(f, x_min, x_max)[1]

(quadgk(f, x_min, x_max))[1] = 0.3333333333333333


0.3333333333333333

In [3]:
using ForwardDiff
# d/dx ∫^x_x0 f(z) dz = f(x)
ForwardDiff.derivative(x->quadgk(f, x_min, x)[1], 0.5)

StackOverflowError: StackOverflowError:

In [4]:
methods(quadgk)

# 2 methods for generic function "quadgk":
[1] quadgk(f, a::T, b::T, c::T...; atol, rtol, maxevals, order, norm) where T in QuadGK at /home/paul/.julia/packages/QuadGK/czbUH/src/adapt.jl:177
[2] quadgk(f, a, b, c...; kws...) in QuadGK at /home/paul/.julia/packages/QuadGK/czbUH/src/adapt.jl:173

In [5]:
function helper(x)
  @show typeof(x)
  @show x isa ForwardDiff.Dual
  return x #quadgk(f, x_min, x)[1]  
end
ForwardDiff.derivative(helper, 0.5)

typeof(x) = ForwardDiff.Dual{ForwardDiff.Tag{typeof(helper), Float64}, Float64, 1}
x isa ForwardDiff.Dual = true


1.0

In [6]:
import QuadGK: quadgk
function quadgk(f, a::T, b::ForwardDiff.Dual{D}, c::T...;kws...) where T where D
    out = ForwardDiff.Dual{D}(ForwardDiff.value(quadgk(f, a, ForwardDiff.value(b), c...;kws...)[1]), 
            f(ForwardDiff.value(b))*ForwardDiff.partials(b))
    return(out)
end

quadgk (generic function with 3 methods)

In [8]:
ForwardDiff.derivative(x->exp(quadgk(f, x_min, x))[1], 0.5)


0.26063672629749784

## Trapezoidal Rule (uniform and non-uniform)
$$ \int_{x_{\min}}^{x_{\max}} f(x) dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$

In [3]:
function trapz_integral(f, nodes::AbstractRange)
    M = length(nodes)
    Δ = step(nodes)
    total = zero(eltype(nodes))
    for (i, x) in enumerate(nodes)
        weight = ((i == 1) || (i == M)) ?  Δ/2 : Δ   # ternary operation, Condition ? if true : if false
        total += weight * f(x)
    end
    return total
end

trapz_integral (generic function with 1 method)

In [5]:
trapz_integral(f, x)

0.33335000000000015

In [6]:
function trapz_weights(nodes::AbstractArray)  # Note multiple dispatch
    println("trapz for AbstractArray:")
    M = length(nodes)
    Δ = diff(nodes)
    prepend!(Δ, NaN) # To keep the indexing straight. Now, Δ[2] = Δ_2 = z_2 - z_1. And NaN will throw an error if we try to use it.
    interiorWeights = [(Δ[i] + Δ[i+1])/2 for i = 2:M-1]
    return [Δ[2]/2; interiorWeights; Δ[M]/2]
end

trapz_weights (generic function with 1 method)

In [7]:
w = trapz_weights(x_array)  # call with array
@show w ⋅ f.(x_array);

trapz for AbstractArray:
w ⋅ f.(x_array) = 0.33335


In [9]:
w = trapz_weights(x) # call with range
@show w ⋅ f.(x);

trapz for AbstractArray:
w ⋅ f.(x) = 0.33335


In [10]:
# Specialization.  New method added to `trapz_weights`
function trapz_weights(nodes::AbstractRange)
    println("trapz for AbstractRange:")
    M = length(nodes)
    Δ = step(nodes)
    return [Δ/2; Δ*ones(M-2); Δ/2]
end

trapz_weights (generic function with 2 methods)

In [11]:
w = trapz_weights(x)
@show w ⋅ f.(x);

trapz for AbstractRange:
w ⋅ f.(x) = 0.33335000000000004


## Example: Legendre Quadrature
See https://en.wikipedia.org/wiki/Gaussian_quadrature#Gauss%E2%80%93Legendre_quadrature
$$ \int_{-1}^1 f(x) dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$


$$ \int_{-\infty}^\infty f(x) e^{-x^2} dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$


In [14]:
N = 10
f(x) = x^10 + x^2 + 1
x, w = gausslegendre(N)  # can adjust to change range
@show quadgk(f, -1.0, 1.0)[1]
dot(w, f.(x)) # note few evaluations

(quadgk(f, -1.0, 1.0))[1] = 2.8484848484848486


2.848484848484849